In [32]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

# Original full feature engineering function
def engineer_features_no_pca(df):
    df = df.copy()
    new_cols = {}

    # Weighted component-property features
    for comp in range(1, 6):
        for prop in range(1, 11):
            prop_col = f'Component{comp}_Property{prop}'
            frac_col = f'Component{comp}_fraction'
            new_cols[f'{prop_col}_weighted'] = df[prop_col] * df[frac_col]

    # Interaction features between fractions
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]
            new_cols[f'{frac_i}_div_{frac_j}'] = df[frac_i] / (df[frac_j] + 1e-8)

    # Ratios to total fraction
    total_fraction = sum(df[f'Component{i}_fraction'] for i in range(1, 6))
    new_cols['Total_fraction'] = total_fraction
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_to_total'] = df[frac_col] / (total_fraction + 1e-8)

    # Aggregate stats for component properties
    for comp in range(1, 6):
        props = [f'Component{comp}_Property{p}' for p in range(1, 11)]
        new_cols[f'Component{comp}_prop_mean'] = df[props].mean(axis=1)
        new_cols[f'Component{comp}_prop_std'] = df[props].std(axis=1)
        new_cols[f'Component{comp}_prop_min'] = df[props].min(axis=1)
        new_cols[f'Component{comp}_prop_max'] = df[props].max(axis=1)
        new_cols[f'Component{comp}_prop_range'] = new_cols[f'Component{comp}_prop_max'] - new_cols[f'Component{comp}_prop_min']

    # Polynomial features for component fractions (squares)
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_squared'] = df[frac_col] ** 2

    # Cross product polynomial features
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]

    new_features_df = pd.DataFrame(new_cols, index=df.index)
    df = pd.concat([df, new_features_df], axis=1)
    return df

# Use top 20 total gain features
TOP_20_FEATURES = [
    "Component2_fraction",
    "Component2_fraction_div_Component4_fraction",
    "Component1_fraction_x_Component2_fraction",
    "Component1_Property7_weighted",
    "Component3_Property7_weighted",
    "Component2_Property7",
    "Component2_Property7_weighted",
    "Component3_Property7",
    "Component1_Property7",
    "Component2_fraction_div_Component5_fraction",
    "Component2_Property8_weighted",
    "Component2_fraction_div_Component3_fraction",
    "Component4_Property7_weighted",
    "Component4_Property5_weighted",
    "Component2_Property8",
    "Component1_fraction_div_Component5_fraction",
    "Component4_fraction_x_Component5_fraction",
    "Component1_fraction",
    "Component4_Property7",
    "Component1_fraction_div_Component4_fraction"
]

# Filter to just the top 20
def engineer_features_reduced(df):
    df_full = engineer_features_no_pca(df)
    return df_full[TOP_20_FEATURES]

# Load data
X = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_train.csv')
y = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/y_train.csv')
X_test = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_test.csv')

# Feature engineering with only top 20
X_fe = engineer_features_reduced(X)
X_test_fe = engineer_features_reduced(X_test)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_fe, y, test_size=0.2, random_state=42)

# Choose target
target_index = 2
print(f"Training model for target {target_index + 1}")

# Train the model
model = xgb.XGBRegressor(
    max_depth=5,
    n_estimators=100000,
    learning_rate=0.05,
    reg_alpha=0.95,
    reg_lambda=0,
    eval_metric='mae',
    random_state=42,
    early_stopping_rounds=25
)

model.fit(
    X_train, y_train.iloc[:, target_index],
    eval_set=[(X_val, y_val.iloc[:, target_index])],
    verbose=50
)

# Evaluate
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)
test_preds = model.predict(X_test_fe)

train_mape = mean_absolute_percentage_error(y_train.iloc[:, target_index], train_preds)
val_mape = mean_absolute_percentage_error(y_val.iloc[:, target_index], val_preds)

print(f"\nTrain MAPE for target {target_index + 1}: {train_mape:.4f}")
print(f"Validation MAPE for target {target_index + 1}: {val_mape:.4f}")

# Save predictions
output_df = pd.DataFrame(test_preds, columns=[f'Target{target_index + 1}_prediction'])
output_df.to_csv('target_3_test_predictions.csv', index=False)
print("Test predictions saved to 'target_3_test_predictions.csv'")




Training model for target 3
[0]	validation_0-mae:0.76464
[50]	validation_0-mae:0.21021
[100]	validation_0-mae:0.15470
[150]	validation_0-mae:0.14259
[200]	validation_0-mae:0.13720
[250]	validation_0-mae:0.13502
[300]	validation_0-mae:0.13317
[350]	validation_0-mae:0.13198
[400]	validation_0-mae:0.13140
[442]	validation_0-mae:0.13116

Train MAPE for target 3: 0.4352
Validation MAPE for target 3: 0.7229
Test predictions saved to 'target_3_test_predictions.csv'
